# initatization 

In [1]:
import pandas as pd
import numpy as np
import os
import tarfile
import netCDF4 as nc
import xarray as xr
from glob import glob 
import rasterio
import functools
from functools import reduce

In [2]:
import shapely
import fiona 
import pyproj
import geopandas as gpd

In [3]:
debris_dat = pd.read_csv('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/rgi60_points_gt04km2_debrisstats.csv')

In [4]:
climate_stat_13 = pd.read_csv('climate_statistics_13.csv')
climate_stat_14 = pd.read_csv('climate_statistics_14.csv')
climate_stat_15 = pd.read_csv('climate_statistics_15.csv')
glacier_stat_13 = pd.read_csv('glacier_statistics_13.csv')
glacier_stat_14 = pd.read_csv('glacier_statistics_14.csv')
glacier_stat_15 = pd.read_csv('glacier_statistics_15.csv')
huggonet_dat = pd.read_csv('hugonnet_2021_ds_rgi60_pergla_rates_10_20_worldwide.csv')
climate_historical = pd.read_csv('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/climate_historical.csv')
ice_thickness_dat = pd.read_csv('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/ice_thickness_df.csv')
debris_dat = pd.read_csv('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/rgi60_points_gt04km2_debrisstats.csv')
lakes_dat = pd.read_csv('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/supraglacial_lakes_df.csv')
rgi13 = gpd.read_file('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/RGI/nsidc0770_13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp')
rgi14 = gpd.read_file('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/RGI/nsidc0770_14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp')
rgi15 = gpd.read_file('/Users/bovandenbosch/Documents/Master_Applied Data_Science/Thesis/Data/RGI/nsidc0770_15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')

In [5]:
pd.set_option('display.max_columns', None)

In [7]:
# hoeveel glaciers bestuderen wij:
area_bigger_2 = (len(glacier_stat_13[glacier_stat_13['rgi_area_km2']>2])+
 len(glacier_stat_14[glacier_stat_14['rgi_area_km2']>2])+
 len(glacier_stat_15[glacier_stat_15['rgi_area_km2']>2]))

tot = (len(glacier_stat_13)+
 len(glacier_stat_14)+
 len(glacier_stat_15))

(area_bigger_2/tot)*100

8.477432590855804

# Climate Statistics

In [8]:
climate_stat = pd.concat([climate_stat_13,climate_stat_14,climate_stat_15],ignore_index = True)

In [9]:
(not any(climate_stat['glacier_type'] == 'Glacier'), #checking if all glacier types are glaciers
 not any(climate_stat['status'] == 'Glacier or ice cap'), 
 not any(climate_stat['terminus_type'] == 'Land-terminating'))

(False, False, False)

In [10]:
climate_stat = climate_stat[['rgi_id','tstar_aar','tstar_ela_h']]

In [11]:
climate_stat.tail()

,rgi_id,tstar_aar,tstar_ela_h
95531,RGI60-15.13115,0.518153,5647.349262
95532,RGI60-15.13116,0.537703,5613.211517
95533,RGI60-15.13117,0.580946,5977.445401
95534,RGI60-15.13118,0.465043,5994.382197
95535,RGI60-15.13119,0.571124,5834.455509


In [12]:
climate_stat.isnull().sum(axis=0)

rgi_id          0
tstar_aar      89
tstar_ela_h    89
dtype: int64

In [13]:
climate_stat['tstar_aar'].isna().sum()/len(climate_stat['tstar_aar'])

0.0009315859989951432

# Glacier Statistics

In [14]:
glacier_stat = pd.concat([glacier_stat_13,glacier_stat_14,glacier_stat_15])

In [15]:
(not any(glacier_stat['glacier_type'] == 'Glacier'), #checking: all glacier_types are glaciers
not any(glacier_stat['terminus_type'] == 'Land-terminating'))

(False, False)

In [16]:
glacier_stat = glacier_stat[['rgi_id','rgi_area_km2']]

In [17]:
glacier_stat.head()

,rgi_id,rgi_area_km2
0,RGI60-13.00001,0.432
1,RGI60-13.00002,0.367
2,RGI60-13.00003,0.070
3,RGI60-13.00004,0.255
4,RGI60-13.00005,0.261


In [18]:
glacier_stat.isnull().sum(axis=0)

rgi_id          0
rgi_area_km2    0
dtype: int64

# Huggonet

In [19]:
# remove time periode 2000-2020
huggonet_dat = huggonet_dat[huggonet_dat["period"].str.contains("2000-01-01_2020-01-01")==False]
huggonet_dat = huggonet_dat.rename(columns={"rgiid": "rgi_id"})

In [20]:
huggonet_dat = huggonet_dat[['rgi_id','period','dmdtda','dvoldt','dhdt']]

In [21]:
huggonet_dat.head()

,rgi_id,period,dmdtda,dvoldt,dhdt
1,RGI60-01.00001,2010-01-01_2020-01-01,-0.0472,-20003.0,-0.0556
2,RGI60-01.00001,2000-01-01_2010-01-01,0.0217,9175.0,0.0255
4,RGI60-01.00002,2010-01-01_2020-01-01,-0.2898,-190257.0,-0.3410
5,RGI60-01.00002,2000-01-01_2010-01-01,-0.1683,-110465.0,-0.1980
7,RGI60-01.00003,2010-01-01_2020-01-01,-1.0428,-2067231.0,-1.2268


In [22]:
huggonet_dat.isnull().sum(axis=0)

rgi_id        0
period        0
dmdtda    16020
dvoldt    16020
dhdt      14120
dtype: int64

# climate historical 2000-2020

In [23]:
# make climate historical same time formate as hugonet
climate_historical['time'] = climate_historical['time'].replace({'2009-12-31':'2000-01-01_2010-01-01', '2019-12-31':'2010-01-01_2020-01-01'})
climate_historical = climate_historical.rename(columns={"time": "period"})


In [24]:
climate_historical.tail()

,period,prcp,temp,rgi_id
191067,2010-01-01_2020-01-01,5144.9110,-4.920659,RGI60-15.13117
191068,2000-01-01_2010-01-01,4942.9070,-5.594553,RGI60-15.13118
191069,2010-01-01_2020-01-01,5144.9110,-4.920659,RGI60-15.13118
191070,2000-01-01_2010-01-01,5431.8496,-6.466131,RGI60-15.13119
191071,2010-01-01_2020-01-01,5656.5050,-5.735077,RGI60-15.13119


In [25]:
climate_historical.isnull().sum(axis=0)

period    0
prcp      0
temp      0
rgi_id    0
dtype: int64

# Debris Data

In [26]:
debris_dat

,RGIId,lon,lat,tot_area,deb_area,cln_area,tot_vol,deb_vol,cln_vol,tota_ela,deba_ela,totv_ela,deb_v_ela,area_perc,vol_perc,vol_ela_p
0,RGI60-13.00001,79.4939,34.4713,431100,11700,419400,2.186700e+06,3.055613e+03,2.183644e+06,99000,3600,3.257255e+05,5.100403e+02,2.713990,0.139736,0.156586
1,RGI60-13.00014,78.0681,35.5749,649800,9000,640800,2.556373e+06,2.557897e+02,2.556117e+06,128700,8100,3.725382e+05,2.426350e+02,1.385040,0.010006,0.065130
2,RGI60-13.00015,78.4388,35.5637,934200,127800,806400,9.657947e+06,1.275462e+06,8.382486e+06,73800,4500,5.389982e+05,1.150888e+04,13.680200,13.206300,2.135240
3,RGI60-13.00017,78.3496,35.5606,1320300,108000,1212300,1.997695e+07,2.685162e+06,1.729179e+07,159300,15300,1.031575e+06,9.246398e+03,8.179960,13.441300,0.896338
4,RGI60-13.00020,78.1002,35.5632,439200,9900,429300,1.998172e+06,1.981467e+03,1.996191e+06,72000,5400,3.328289e+05,1.662830e+03,2.254100,0.099164,0.499605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33582,RGI60-15.13097,80.2200,30.5950,3611700,3600,3608100,1.595523e+08,4.482747e+03,1.595478e+08,2248200,1800,1.150574e+08,4.177935e+03,0.099676,0.002810,0.003631
33583,RGI60-15.13104,79.4560,31.0520,22709700,1344600,21365100,2.727866e+09,1.429210e+08,2.584945e+09,6532200,1334700,1.048977e+09,1.428911e+08,5.920820,5.239300,13.621900
33584,RGI60-15.13115,81.9770,30.3370,1355400,51300,1304100,2.898304e+07,1.112262e+05,2.887181e+07,729900,25200,1.939320e+07,6.671299e+04,3.784860,0.383763,0.344002
33585,RGI60-15.13116,81.9910,30.3390,7429500,264600,7164900,7.816793e+08,2.531266e+06,7.791480e+08,3733200,252000,4.656625e+08,2.436067e+06,3.561480,0.323824,0.523140


In [27]:
debris_dat = debris_dat.rename(columns={"RGIId": "rgi_id","vol_ela_p":"debris_vol_ela_p"})

In [28]:
debris_dat['debris_area_ela_p'] = (debris_dat['deba_ela']/debris_dat['tota_ela'])*100

In [29]:
debris_dat = debris_dat[['rgi_id','debris_vol_ela_p','debris_area_ela_p']]




In [30]:
debris_dat

,rgi_id,debris_vol_ela_p,debris_area_ela_p
0,RGI60-13.00001,0.156586,3.636364
1,RGI60-13.00014,0.065130,6.293706
2,RGI60-13.00015,2.135240,6.097561
3,RGI60-13.00017,0.896338,9.604520
4,RGI60-13.00020,0.499605,7.500000
...,...,...,...
33582,RGI60-15.13097,0.003631,0.080064
33583,RGI60-15.13104,13.621900,20.432626
33584,RGI60-15.13115,0.344002,3.452528
33585,RGI60-15.13116,0.523140,6.750241


# RGI Data

In [31]:
rgi_concat = pd.concat([rgi13,rgi14,rgi15],ignore_index=True)
rgi_concat = rgi_concat.rename(columns={"RGIId": "rgi_id"})

In [32]:
rgi_concat = pd.concat([rgi13,rgi14,rgi15],ignore_index=True)
rgi_concat = rgi_concat.rename(columns={"RGIId": "rgi_id"})
rgi = rgi_concat[['rgi_id','O2Region','Slope','Aspect','Lmax','Area','Zmax','Zmin','Zmed']]
rgi_df = pd.DataFrame(rgi)
rgi_df.to_csv('rgi_df_pd_format')

In [33]:
rgi_df.isnull().sum(axis=0)

rgi_id      0
O2Region    0
Slope       0
Aspect      0
Lmax        0
Area        0
Zmax        0
Zmin        0
Zmed        0
dtype: int64

# Supraglacier Lakes

In [34]:
lakes = lakes_dat.rename(columns={"RGIId": "rgi_id"})
lakes = lakes[['rgi_id']]
lakes['glacial_lake'] = 1
lakes = rgi_df.merge(lakes, on = 'rgi_id', how = 'outer')
lakes = lakes[['rgi_id','glacial_lake']].fillna(0)
lakes

,rgi_id,glacial_lake
0,RGI60-13.00001,1.0
1,RGI60-13.00002,0.0
2,RGI60-13.00003,0.0
3,RGI60-13.00004,0.0
4,RGI60-13.00005,0.0
...,...,...
95929,RGI60-15.13115,1.0
95930,RGI60-15.13116,0.0
95931,RGI60-15.13117,0.0
95932,RGI60-15.13118,0.0


In [35]:
lakes.isnull().sum(axis=0)

rgi_id          0
glacial_lake    0
dtype: int64

# Ice Thickness

In [36]:
ice_thickness_dat

,Unnamed: 0,rgi_id,ice_thickness
0,0,RGI60-13.00001,33.668630
1,1,RGI60-13.00002,39.834972
2,2,RGI60-13.00003,14.957049
3,3,RGI60-13.00004,23.991674
4,4,RGI60-13.00005,20.455612
...,...,...,...
95529,95529,RGI60-15.13115,36.147495
95530,95530,RGI60-15.13116,90.726250
95531,95531,RGI60-15.13117,110.223430
95532,95532,RGI60-15.13118,14.799946


In [37]:
ice_thickness_dat = ice_thickness_dat[['rgi_id','ice_thickness']]

In [38]:
ice_thickness_dat.tail()

,rgi_id,ice_thickness
95529,RGI60-15.13115,36.147495
95530,RGI60-15.13116,90.726250
95531,RGI60-15.13117,110.223430
95532,RGI60-15.13118,14.799946
95533,RGI60-15.13119,12.369187


In [39]:
ice_thickness_dat.isnull().sum(axis=0)

rgi_id           0
ice_thickness    0
dtype: int64

# Dataframe 

In [40]:
## climate_stat
# glacier_stat HOEFT NIET 
## huggonet_dat & climate_historical = merge-clim_huggo
## ice_thickness_dat  
## debris_dat # only contains dat >0.4 km2 
## lakes
## rgi_df

In [41]:
# join
# huggonet: rgiid & period
# climate_historical: rgi_id & time 

merge_clim_huggo = pd.merge(huggonet_dat,climate_historical, on=['rgi_id','period'])

In [42]:
merge_clim_huggo.isnull().sum(axis=0)

rgi_id       0
period       0
dmdtda    3722
dvoldt    3722
dhdt      3722
prcp         0
temp         0
dtype: int64

In [43]:
merge_clim_huggo['dvoldt'].isnull().sum(axis=0)/len(merge_clim_huggo['dvoldt'])

0.019479567911572602

In [44]:
(climate_stat.shape, glacier_stat.shape, merge_clim_huggo.shape,ice_thickness_dat.shape, debris_dat.shape, rgi_df.shape)


((95536, 3), (95536, 2), (191072, 7), (95534, 2), (33587, 3), (95536, 9))

In [45]:
df = merge_clim_huggo.merge(climate_stat,on='rgi_id',how='inner').merge(rgi_df, on='rgi_id',how='inner').merge(ice_thickness_dat, on = 'rgi_id', how='inner')
df.shape

(191068, 18)

In [46]:
df_all = df.merge(lakes, on = 'rgi_id', how= 'outer')
df.shape

(191068, 18)

In [47]:
df_all = df

In [48]:
df = df[df['Area'] > 0.4]
df.shape

(67174, 18)

In [49]:
df = df.merge(debris_dat, on='rgi_id',how='outer')
df.shape

(67174, 20)

In [50]:
df = df[df['Area'] > 2]

In [51]:
df

,rgi_id,period,dmdtda,dvoldt,dhdt,prcp,temp,tstar_aar,tstar_ela_h,O2Region,Slope,Aspect,Lmax,Area,Zmax,Zmin,Zmed,ice_thickness,debris_vol_ela_p,debris_area_ela_p
42,RGI60-13.00062,2000-01-01_2010-01-01,0.2511,656305.0,0.2954,2662.2944,-13.254577,0.622839,5777.281395,5,21.5,358,1532,2.222,6011,5417,5839,47.280240,0.063558,3.950617
43,RGI60-13.00062,2010-01-01_2020-01-01,-0.1517,-396462.0,-0.1784,2571.0835,-13.101825,0.622839,5777.281395,5,21.5,358,1532,2.222,6011,5417,5839,47.280240,0.063558,3.950617
58,RGI60-13.00093,2000-01-01_2010-01-01,0.3403,808356.0,0.4004,2471.3232,-14.763111,0.349641,5766.503904,5,25.6,50,1219,2.019,6095,5432,5795,31.963816,0.384015,2.211302
59,RGI60-13.00093,2010-01-01_2020-01-01,-0.2084,-494898.0,-0.2451,2483.2847,-14.639121,0.349641,5766.503904,5,25.6,50,1219,2.019,6095,5432,5795,31.963816,0.384015,2.211302
74,RGI60-13.00130,2000-01-01_2010-01-01,0.2993,858801.0,0.3521,2471.3232,-14.763111,0.482537,5864.392600,5,26.4,346,2145,2.439,6308,5428,5864,36.834835,0.018882,1.470588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67167,RGI60-15.13104,2010-01-01_2020-01-01,-0.3959,-10575925.0,-0.4658,7035.4336,-10.042706,0.551624,5805.561280,1,12.2,341,11899,22.704,6752,5235,5834,109.057915,13.621900,20.432626
67170,RGI60-15.13116,2010-01-01_2020-01-01,-0.8608,-7521207.0,-1.0127,9643.4520,-7.883843,0.537703,5613.211517,1,8.9,330,5779,7.427,6232,5266,5633,90.726250,0.523140,6.750241
67171,RGI60-15.13116,2000-01-01_2010-01-01,-0.8456,-7388231.0,-0.9948,9380.2680,-8.199018,0.537703,5613.211517,1,8.9,330,5779,7.427,6232,5266,5633,90.726250,0.523140,6.750241
67172,RGI60-15.13117,2000-01-01_2010-01-01,-0.5257,-4198006.0,-0.6184,4942.9070,-5.594553,0.580946,5977.445401,1,6.3,96,5519,6.788,6147,5696,5994,110.223430,0.011779,0.450007


# General stats

Justify deleting all glaciers <0.4km2

In [57]:
df1 = rgi_df.merge(ice_thickness_dat, on = 'rgi_id', how = 'inner')

In [58]:
df1["volume"] = df1["Area"] * df1["ice_thickness"]

In [59]:
# count percentage of all glaicer >0.4
(len(df1[df1['Area']> 2])/len(df1))*100

8.47761006552641

In [60]:
# count percentage of all glacier <0.4
(len(df1[df1['Area']<2])/len(df1)) *100

91.52029643896414

In [61]:
# volume percentage of all glaciers >0.4 km2
(df1[df1['Area'] > 2]['volume'].sum()/(df1['volume'].sum()))*100

86.38387386123219

In [62]:
# volume percentage of all glaciers > 0.4 km2
(df1[df1['Area'] < 2]['volume'].sum()/(df1['volume'].sum()))*100

13.613219894605086